In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 

# Загрузка данных

In [2]:
train = pd.read_parquet('data_fusion_train.parquet', engine='pyarrow')
train = train[train.category_id != -1].drop_duplicates('item_name')

In [3]:
other_data = pd.read_parquet('data_fusion_train.parquet', engine='pyarrow')
other_data = other_data[other_data.category_id == -1].drop_duplicates('item_name')

# Размеченные данные

In [4]:
train

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
1,11,6,20:34,"Молоко 3,2%,шт",2.000,8,2,78,None
3,39,4,11:28,"Компот из изюма, 114 ккал",1.000,4,1,71,None
4,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.000,4,1,71,None
17,56,5,11:42,Кофе Капучино Большой Эден 18,1.000,12,1,70,None
40,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.000,7,-1,84,None
...,...,...,...,...,...,...,...,...,...
45669181,9880594,2,20:11,"Напиток Энерг. Ред Булл 0,355л",1.000,10,6,83,None
45681543,9908635,5,01:09,Хеменгуэй Дайкири,1.000,15,6,0,None
45690702,9929539,0,14:39,"Пиво светлое ""Халзан"" 4,5 % об, пл/б. 1,5 л(шт)",1.000,10,6,0,None
45692298,9932986,3,22:26,Экспресс педикюр,1.000,15,6,42,None


# Не размеченные данные

In [5]:
other_data

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,7,5,09:50,Флуконазол капс 150мг №1 Вертекс,1.000,7,-1,-1,вертекс
2,39,4,11:28,"Борщ Станичный с тушенкой, 103 ккал (завод, з/у)",1.000,6,1,-1,None
6,39,4,11:28,"Филе Горизонт (филе птицы), 218 ккал",1.000,7,1,-1,горизонт
10,49,3,14:27,3 Бумага Упак 1/Прочи/В асс,1.000,8,1,-1,None
11,49,3,14:27,4 Бумага Упак 1/Прочи/В асс,1.000,8,1,-1,None
...,...,...,...,...,...,...,...,...,...
45729191,10016000,1,13:07,Краски аквар.12 цвет.медовые BRAUBERG,1.000,8,6,-1,brauberg
45729194,10016000,1,13:07,Зонт мужской автомат <>,1.000,16,6,-1,None
45729196,10016004,2,18:43,"Пиво Ячменное фирменное с/б 0,5 л",2.000,9,-1,-1,None
45729312,10016159,4,12:39,Слива Черная Пантера 1кг,0.439,11,-1,-1,None


# \*\*\* Изучаем поле item_name \*\*\*

## Есть ли дубликаты в названиятоваров? Нет!

In [10]:
len(other_data['item_name'].tolist()) - len(set(other_data['item_name'].tolist()))

0

## Есть ли повторяющиеся слова? Точно есть! Но какие?